In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
df_hut = pd.read_csv("http://www.bcn.cat/huts/hut_comunicacio_opendata.csv")
df_hut.head(5)

,N_EXPEDIENT,DISTRICTE,BARRI,TIPUS_CARRER,CARRER,TIPUS_NUM,NUM1,LLETRA1,NUM2,LLETRA2,BLOC,PORTAL,ESCALA,PIS,PORTA,NUMERO_REGISTRE_GENERALITAT,NUMERO_PLACES,LONGITUD_X,LATITUD_Y
0,01-2013-0753,CIUTAT VELLA,el Raval,Carrer,HOSPITAL,1,25,NaN,NaN,NaN,NaN,NaN,NaN,1,2,HUTB-000715,9.0,2.171125,41.379035
1,02-2014-0566,L'EIXAMPLE,la Sagrada Família,Avinguda,GAUDI,1,51,NaN,53.0,NaN,NaN,NaN,NaN,EN,1,HUTB-010412,7.0,2.173113,41.406595
2,02-2014-0207,L'EIXAMPLE,la Dreta de l'Eixample,Plaça,TETUAN,1,33,NaN,NaN,NaN,NaN,NaN,NaN,SA,2,HUTB-008754,4.0,2.174958,41.393788
3,01-2014-0073,CIUTAT VELLA,el Raval,Carrer,JUNTA DE COMERÇ,1,23,NaN,NaN,NaN,NaN,NaN,NaN,2,1A,HUTB-013228,4.0,2.170788,41.377598
4,02-2014-1125,L'EIXAMPLE,la Dreta de l'Eixample,Gran Via,CORTS CATALANES,2,682,NaN,NaN,NaN,NaN,NaN,NaN,2,1,HUTB-011029,8.0,2.171954,41.390836


In [3]:
df_hut['BARRI'].value_counts()

la Dreta de l'Eixample             1653
la Sagrada Família                  757
la Vila de Gràcia                   734
l'Antiga Esquerra de l'Eixample     693
Sant Antoni                         580
                                   ... 
les Roquetes                          2
la Vall d'Hebron                      1
la Guineueta                          1
el Bon Pastor                         1
Canyelles                             1
Name: BARRI, Length: 64, dtype: int64